<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 从头实现BPE分词器
- 这是一个独立的notebook，从零开始实现流行的字节对编码（BPE）分词算法，该算法用于 GPT-2 到 GPT-4、Llama 3 等模型
- 关于分词目的的更多详情，请参考[第2章](https://github.com/zzfive/LLMs-from-scratch-bias/blob/main/ch02/01_main-chapter-code/ch02.ipynb)；这里的代码是解释 BPE 算法的额外材料
- OpenAI 为训练原始 GPT 模型实现的原始 BPE 分词器可以在[这里](https://github.com/openai/gpt-2/blob/master/src/encoder.py)找到
- BPE 算法最初是由 Philip Gage 在 1994 年的论文"[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"中描述
- 现在大多数项目，包括 Llama 3，都使用 OpenAI 的开源 [tiktoken 库](https://github.com/openai/tiktoken)，因为它的计算性能；例如，它允许加载预训练的 GPT-2 和 GPT-4 分词器（Llama 3 模型也是使用 GPT-4 分词器训练的）
- 上述实现与本笔记本中的实现之间的区别，除了它之外，还包括一个用于训练分词器的函数（出于教育目的）
- 还有一个名为 [minBPE](https://github.com/karpathy/minbpe) 的实现，它支持训练，可能更高效（这里的实现主要关注教育目的）；与 `minbpe` 相比，此处的实现还允许加载原始 OpenAI 分词器词汇表和 BPE "合并"（此外，Hugging Face 分词器也能够训练和加载各种分词器；更多信息请参见[这个 GitHub 讨论](https://github.com/rasbt/LLMs-from-scratch/discussions/485)，这是一位读者在尼泊尔语上训练 BPE 分词器的讨论）

&nbsp;
# BPE背后的主要思想
- BPE的主要想法是将文本转换为可用于LLM训练的整数表征，即token ID，如[Chapter 2](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)中所示

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/bpe-overview.webp" width="600px">
</p>

&nbsp;
## 1.1 Bits and bytes/位与字节
- 在开始BPE之前，先引入字节的概念
- 考虑将文本转换为字节数组（毕竟BPE代表"字节"对编码）

In [21]:
text = "This is some text"
byte_array = bytearray(text, "utf-8")
print(byte_array)

bytearray(b'This is some text')


- 对bytearray对象调用list()，会每个字节被视为单独的元素，结果是一个与字节值对应的整数列表

In [22]:
ids = list(byte_array)  # 每一个字节对应一个字符，每个字节内存存储一个数值，唯一表示该字符
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


In [24]:
# 可以使用chr函数查看一个字节数值对应的字符
chr(105)

'i'

- 这是一种有效的方法，可以将文本转换为在大型语言模型(LLM)嵌入层中需要的token ID表示
- 然而，这种方法的缺点是它为每个字符创建一个ID（对于短文本来说，这会产生很多ID）
- 也就是说，对于一个17个字符的输入文本，必须使用17个标记ID作为LLM的输入

In [25]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


- 如果你之前使用过大型语言模型(LLM)，你可能知道BPE分词器有一个词汇表，为整个单词或子词分配token ID，而不是为每个字符分配token ID。
- 例如，GPT-2分词器将相同的文本（"This is some text"）仅标记为4个标记而不是17个：1212, 318, 617, 2420
- 可以使用交互式[tiktoken应用](https://tiktokenizer.vercel.app/?model=gpt2)或[tiktoken库](https://github.com/openai/tiktoken)来验证这一点：

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">
</p>

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```


- 因为一个字节包含8个bits，有2<sup>8</sup> = 256种可能的值，即一个字节可表示256不同的字符，从0~255
- 执行以下代码 `bytearray(range(0, 257))`, 会收到异常 `ValueError: byte must be in range(0, 256)`)
- BPE分词器通常使用这256个值作为其前256个**单字符标记**；可以通过运行以下代码进行直观检查：

```python
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")
"""
prints:
0: !
1: "
2: #
...
255: �  # <---- single character tokens up to here
256:  t
257:  a
...
298: ent
299:  n
"""
```

- 上面需要注意的是，第256和257项不是单字符值，而是双字符值（一个空格+一个字母），这是原始GPT-2 BPE分词器的一个小缺点（在GPT-4分词器中已经得到改进）

## 1.2 建立词汇表
- BPE标记化算法的目标是构建一个包含常见子词的词汇表，如298: ent（例如可以在entangle, entertain, enter, entrance, entity, ...等词中找到），甚至是完整的单词

```
318: is
617: some
1212: This
2420: text
```

- 在开始进行实际代码实现之前，如今用于LLM分词器的形式可以总结如下

## 1.3 BPE算法概述

**1. 识别频繁对**
- 在每次迭代中，扫描文本以找到最常出现的字节对（或字符对）

**2. 替换并记录**
- 用一个新的占位符ID替换该对（使用尚未使用的ID，例如，如果我们从0...255开始，第一个占位符将是256）
- 在查找表中记录这个映射
- 查找表的大小是一个超参数，也称为"词汇表大小"（对于GPT-2，是50,257）

**3. 重复直到没有收益**
- 不断重复步骤1、2，持续合并最频繁的对
- 当不能继续压缩后停止

**解压缩/解码**
- 恢复原始文本，使用查找表将每个ID替换为其对应的对，从而逆转该过程

&nbsp;
## 1.4 BPE算法例子

### 1.4.1 上述步骤1、2的具体例子
- 假设有文本(训练数据) "the cat in the hat"，要从中为BPE分词器构建词汇表
**迭代1**
1. 识别频繁对
- 在这个文本中，"th"出现了两次
2. 替换并记录
- 用一个尚未使用的新标记ID替换"th"，例如，256
- 新文本是："<256>e cat in <256>e hat"
- 新词汇表是

```
  0: ...
  ...
  256: "th"
```

**迭代2**
1. 识别频繁对
- 在文本<256>e cat in <256>e hat中，对<256>e出现了两次
2. 替换并记录
- 用一个尚未使用的新标记ID替换"<256>e"，例如，257
- 新文本是："<257> cat in <257> hat"
- 新词汇表是

```
  0: ...
  ...
  256: "th"
  257: "<256>e"
```

**迭代3**
1. 识别频繁对
- 在文本<257> cat in <257> hat中，对<257> 出现了两次
2. 替换并记录
- 用一个尚未使用的新标记ID替换"<257> "，例如，258
- 新文本是："<258>cat in <258>hat"
- 新词汇表是

```
  0: ...
  ...
  256: "th"
  257: "<256>e"
  258: "<257> "
```

**依此类推**

&nbsp;
### 1.4.2 解码部分的具体例子(步骤3)
- 要恢复原始文本，通过按照引入的相反顺序，用每个token ID对应的对来替换它们，从而逆转这个过程
- 从最后的压缩文本开始："<258>cat in <258>hat"
- 代替 `<258>` → `<257> `: `<257> cat in <257> hat`  
- 代替 `<257>` → `<256>e`: `<256>e cat in <256>e hat`
- 代替 `<256>` → "th": `the cat in the hat`

# 2 一个简单的BPE实现
- 下面是上述算法的Python类实现，模仿了tiktoken Python的用户接口
- 请注意，上面的编码部分通过train()描述了原始训练步骤；然而，encode()方法的工作方式类似（尽管由于特殊标记处理看起来更复杂）:
1. 将输入文本拆分为单个字节
2. 重复查找并替换（合并）相邻标记（对），当它们匹配学习到的BPE合并中的任何对时（从最高到最低"等级"，即按照学习的顺序）
3. 继续合并，直到无法应用更多合并
4. 最终的标记ID列表就是编码输出